# Prededecir recursivo con rolling means por categoria

## Data

In [1]:
from DataLoader import importar_ventas,leer_csv
ventas=importar_ventas()

In [2]:
from DataLoader import importar_ventas,leer_csv
len(leer_csv("eci_product_master")["subgroup"].unique())

74

In [3]:
TARGET = ["total_sales"]
FEATURES_PRIMITIVAS = ['date','day', 'month',
                       "store_cod",'subgroup_cod', 'year',"cluster","mean_price" ]

FEATURES=[
    'subgroup_cod', 'total_sales_lag_1', 'year',
    'anual_sin_1', 'anual_cos_1', 'mensual_sin_1', 'mensual_sin_2',
    'mean_price_rolling_mean_7d', 'mean_price_rolling_sum_14d',
    'total_sales_rolling_mean_7d', 'total_sales_rolling_sum_7d',
    'total_sales_rolling_mean_14d', 'total_sales_rolling_sum_14d'
]

In [33]:
import numpy as np
import pandas as pd
def avanzar_rolling_feature(chunk,buffer,i,sum_column,mean_column,window,feature):
    if type(sum_column)==pd.Series:
        sum_column=sum_column.to_numpy()
    if type(mean_column)==pd.Series:
        mean_column=mean_column.to_numpy()
    
    if i<window:
        buffer[i]=chunk.iloc[i][feature]
        mean_column[i]=buffer.mean()
        sum_column[i]=buffer.sum()
    else:
        buffer=chunk.iloc[i-window:i][feature].to_numpy()
        mean_column[i]=buffer.mean()
        sum_column[i]=buffer.sum()
    return buffer,mean_column,sum_column
        
def generar_rolling_features(df,feature,window,stores,subgroups,stop):
    means=[]
    sums=[]
    final_buffers={}
    df=df.sort_values(by=["subgroup_cod","store_cod","date"])
    for subgroup in subgroups:
        for store in stores: 
            chunk=df[((df["subgroup_cod"]==subgroup )& (df["store_cod"]==store) 
                      &(df["date"]<stop))]
            mean_column=np.zeros(len(chunk))    
            sum_column=np.zeros(len(chunk))
            buffer=np.zeros(window)
            for i in range(len(chunk)):
                buffer , mean_column , sum_column= avanzar_rolling_feature(chunk,buffer, 
                                                                           i, sum_column,
                                                                           mean_column,window,
                                                                           feature)
            final_buffers[(subgroup,store)]=buffer
            means.append(mean_column)
            sums.append(sum_column)
    df[f"{feature}_rolling_mean_{window}d"]=np.concatenate(means)
    df[f"{feature}_rolling_sum_{window}d"]=np.concatenate(sums)
    return df, final_buffers



In [34]:
ventas, final_buffers= generar_rolling_features(ventas,"total_sales",7,range(1,158),range(0,75),"2024-01-01")

KeyboardInterrupt: 

In [4]:
def obtener_buffers(ventas):
    subgroups=ventas["subgroup_cod"].unique()
    stores=ventas["store_cod"].unique()
    buffers={}
    ventas=ventas.sort_values(["subgroup_cod","store_cod","date"])
    for subgroup in subgroups:
        for store in stores:
            data=ventas[((ventas["subgroup_cod"]==subgroup) & (ventas["store_cod"]==store))]
            buffers[(subgroup,store,"mean_price")]= data["mean_price"].tail(14).to_numpy()
            buffers[(subgroup,store,"total_sales")]=data["total_sales"].tail(14).to_numpy()
    return buffers

In [ ]:
buffers=obtener_buffers(ventas)
buffers

In [8]:
from DataLoader import generar_subgroup_cod,generar_store_cod, leer_csv,subgroup_category
import pandas as pd
rolling_features=['mean_price_rolling_mean_7d', 'mean_price_rolling_sum_14d',
    'total_sales_rolling_mean_7d', 'total_sales_rolling_sum_7d',
    'total_sales_rolling_mean_14d', 'total_sales_rolling_sum_14d',"total_sales_lag_1"]
def test_recursivo(rolling_features,buffers):
    test=leer_csv("ids_test.csv")
    test[['store_id', 'subgroup', 'date']] = test["store_subgroup_date_id"].str.split('_', expand=True)
    test["store_cod"]=test["store_id"].map(generar_store_cod())
    test["subgroup_cod"]=test["subgroup"].map(generar_subgroup_cod())
    test["date"]=pd.to_datetime(test["date"])
    test.index=test["date"]
    dia_inicial= pd.to_datetime("2024-01-01")
    iniciar=test.loc[test["date"] == dia_inicial, ["subgroup_cod","store_cod"]]
    
    for feature in rolling_features:
        test[feature]=0.0 
    
    
    for index,row in iniciar.iterrows():
        if row["subgroup_cod"]!=32: #basketball
            buffer_total_sales=buffers[(row["subgroup_cod"],row["store_cod"],"total_sales")]
            buffer_mean_price=buffers[(row["subgroup_cod"],row["store_cod"],"mean_price")]
            for feature in rolling_features:
                if "total_sales" in feature:
                    if "rolling_mean" in feature:
                        if "7" in feature:
                            test.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-7:len(buffer_total_sales)].mean()
                        else:
                            test.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-14:len(buffer_total_sales)].mean()
                    
                    elif "rolling_sum" in feature:
                        if "7" in feature:                   
                            test.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-7:len(buffer_total_sales)].sum()
                        else:
                            test.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-14:len(buffer_total_sales)].sum()
                    elif "lag_1" in feature:
                        test.loc[index,feature]=buffer_total_sales[-1]
                else:
                    if "rolling_mean" in feature:
                        if "7" in feature:
                            test.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-7:len(buffer_mean_price)].mean()
                        else:  
                            test.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-14:len(buffer_mean_price)].mean()
                    else:
                        if "7" in feature:                                           
                            test.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-7:len(buffer_mean_price)].sum()
                        else:
                            test.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-14:len(buffer_mean_price)].sum()
                            
        
        test["category"]=test["subgroup"].map(subgroup_category())
        return test
test=test_recursivo(['mean_price_rolling_mean_7d', 'mean_price_rolling_sum_14d',
    'total_sales_rolling_mean_7d', 'total_sales_rolling_sum_7d',
    'total_sales_rolling_mean_14d', 'total_sales_rolling_sum_14d',"total_sales_lag_1"],buffers)
test

,store_subgroup_date_id,store_id,subgroup,date,store_cod,subgroup_cod,mean_price_rolling_mean_7d,mean_price_rolling_sum_14d,total_sales_rolling_mean_7d,total_sales_rolling_sum_7d,total_sales_rolling_mean_14d,total_sales_rolling_sum_14d,total_sales_lag_1,category
date,,,,,,,,,,,,,,
2024-01-01,S00001_Laptops_2024-01-01,S00001,Laptops,2024-01-01,0,0,289.963492,4081.195778,1307.287143,9151.01,1484.105,20777.47,1264.1,Electronics
2024-01-02,S00001_Laptops_2024-01-02,S00001,Laptops,2024-01-02,0,0,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,Electronics
2024-01-03,S00001_Laptops_2024-01-03,S00001,Laptops,2024-01-03,0,0,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,Electronics
2024-01-04,S00001_Laptops_2024-01-04,S00001,Laptops,2024-01-04,0,0,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,Electronics
2024-01-05,S00001_Laptops_2024-01-05,S00001,Laptops,2024-01-05,0,0,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,Electronics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-03,S00157_Puzzles_2024-01-03,S00157,Puzzles,2024-01-03,156,73,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,Toys
2024-01-04,S00157_Puzzles_2024-01-04,S00157,Puzzles,2024-01-04,156,73,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,Toys
2024-01-05,S00157_Puzzles_2024-01-05,S00157,Puzzles,2024-01-05,156,73,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,Toys


In [4]:
import xgboost as xgb
model=xgb.XGBRegressor()
TARGET = ["total_sales"]
FEATURES=[
    'subgroup_cod', 'total_sales_lag_1', 'year',
    'anual_sin_1', 'anual_cos_1', 'mensual_sin_1', 'mensual_sin_2',
    'mean_price_rolling_mean_7d', 'mean_price_rolling_sum_14d',
    'total_sales_rolling_mean_7d', 'total_sales_rolling_sum_7d',
    'total_sales_rolling_mean_14d', 'total_sales_rolling_sum_14d'
]
train_cat=ventas[["category"]+FEATURES]
for i in range(7):
    test.group_by(["subgroup_cod","store_cod"])
    

,store_subgroup_date_id,store_id,subgroup,date,store_cod,subgroup_cod,mean_price_rolling_mean_7d,mean_price_rolling_sum_14d,total_sales_rolling_mean_7d,total_sales_rolling_sum_7d,total_sales_rolling_mean_14d,total_sales_rolling_sum_14d,total_sales_lag_1
date,,,,,,,,,,,,,
2024-01-01,S00001_Laptops_2024-01-01,S00001,Laptops,2024-01-01,0,0,27.422095,391.014667,170.15,1191.05,144.76,2026.64,53.06
2024-01-02,S00001_Laptops_2024-01-02,S00001,Laptops,2024-01-02,0,0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00
2024-01-03,S00001_Laptops_2024-01-03,S00001,Laptops,2024-01-03,0,0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00
2024-01-04,S00001_Laptops_2024-01-04,S00001,Laptops,2024-01-04,0,0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00
2024-01-05,S00001_Laptops_2024-01-05,S00001,Laptops,2024-01-05,0,0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-03,S00157_Puzzles_2024-01-03,S00157,Puzzles,2024-01-03,156,73,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00
2024-01-04,S00157_Puzzles_2024-01-04,S00157,Puzzles,2024-01-04,156,73,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00
2024-01-05,S00157_Puzzles_2024-01-05,S00157,Puzzles,2024-01-05,156,73,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00


In [ ]:
import numpy as np
from xgboost import XGBRegressor
def add_noise(series, scale=0.02):
    noise = np.random.normal(0, scale, size=len(series))
    return series * (1 + noise)

In [ ]:
import numpy as np
from collections import defaultdict
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

np.random.seed(42)  # reproducibilidad

def add_noise(series, scale=0.02):
    noise = np.random.normal(0, scale, size=len(series))
    return series * (1 + noise)

separador = "category"
bins = ventas[separador].unique()
TARGET = ["total_sales"]
FEATURES = [
    'subgroup_cod', 'total_sales_lag_1', 'year',
    'anual_sin_1', 'anual_cos_1', 'mensual_sin_1', 'mensual_sin_2',
    'mean_price_rolling_mean_7d', 'mean_price_rolling_sum_14d',
    'total_sales_rolling_mean_7d', 'total_sales_rolling_sum_7d',
    'total_sales_rolling_mean_14d', 'total_sales_rolling_sum_14d'
]

for bucket in bins:
    print(f"\n=== Bucket: {bucket} ===")    
    data_bucket = ventas[ventas[separador] == bucket].copy()
    # Quitar outliers
    umbral = np.percentile(data_bucket["total_sales"], 95)
    data_bucket = data_bucket[data_bucket["total_sales"] < umbral]
    
    X_train = data_bucket[FEATURES]
    y_train = data_bucket[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    # Entrenar
    reg = XGBRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=6,
        colsample_bytree=0.8,
        reg_lambda=20,
        reg_alpha=0,
        random_state=42,
        n_jobs=-1,
        early_stopping_rounds=50,
        eval_metric="rmse"
    )

    reg.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=50,
    )
    
    # ==== Predicción recursiva en paralelo ====
    buffers = defaultdict(list)
    for _, row in train.iterrows():
        key = (row["store_cod"], row["subgroup_cod"])
        buffers[key].append(row["total_sales"])

    dias_unicos = sorted(hold_out["date"].unique())
    predicciones = []
    reales = []

    for dia in dias_unicos:
        # Día actual para todos los grupos
        df_dia = hold_out[hold_out["date"] == dia].copy()

        # Recalcular rolling de total_sales
        for idx, row in df_dia.iterrows():
            key = (row["store_cod"], row["subgroup_cod"])
            for feat, (agg_type, window) in ROLLING_WINDOWS.items():
                hist = buffers[key][-window:]
                if len(hist) < window:
                    val = np.nan
                else:
                    val = np.mean(hist) if agg_type == "mean" else np.sum(hist)
                df_dia.at[idx, feat] = val

        # Agregar ruido a rolling de precio
        for feat in FEATURES:
            if "mean_price" in feat:
                df_dia[feat] = add_noise(df_dia[feat])

        # Predecir todo el día
        preds_dia = reg.predict(df_dia[FEATURES])

        # Guardar predicciones y reales
        predicciones.extend(preds_dia)
        reales.extend(df_dia[TARGET].values.flatten())

        # Actualizar buffers con predicciones
        for (store, subgroup), pred_val in zip(df_dia[["store_cod", "subgroup_cod"]].values, preds_dia):
            buffers[(store, subgroup)].append(pred_val)

    # Calcular RMSE global
    rmse = np.sqrt(mean_squared_error(reales, predicciones))
    print("RMSE hold_out global:", rmse)

In [18]:
import numpy as np
noise = np.random.normal(0,0.2 , size=1)
noise

array([0.12861368])